In [1]:
# pip install gurobipy

In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
import math
import scipy.stats

In [4]:
import datetime
import time

In [5]:
# Startzeit der Optimierung bestimmen
st = datetime.datetime.now()

In [6]:
# Parameter

In [7]:
# Defintion der Menge der Produkte
Produktarten = [0]*2
for i in range(0,2):
    Produktarten[i] = "Produkt_" + str(i+1)

In [8]:
# Durchzuführende Schritte der Produkte definieren
Schritte = {
    "Produkt_1": {"Schritt_1", "Schritt_2", "Schritt_3", "Schritt_4", "Schritt_5"},
    "Produkt_2": {"Schritt_2", "Schritt_4", "Schritt_6"}
}

In [9]:
# Anzahl der Modularten im Ausgangszustand
Anzahl = {"Modul_1": 1, "Modul_2": 2, "Modul_3": 1, "Modul_4": 1, "Modul_5": 1, "Modul_6": 0}

In [10]:
# Funktion zur Berechnung der Kosten für die Prozessmodule, max(0,Kosten_Module) über Hilfsvariable sichergestellt
def Kosten_Module (x: int, Modul) -> int:
    return (5000*x) - (5000*Anzahl[Modul])

In [11]:
# Eröffnungskosten für Station k
Eröffnungskosten = {"Station_1": 0, "Station_2": 0, "Station_3": 0, "Station_4": 0, "Station_5": 30000, "Station_6": 30000}

In [12]:
# Prozessmodul, das für die Durchführung von Schritt i benötigt wird
Prozessmodul = {"Schritt_1": "Modul_1", "Schritt_2": "Modul_2", "Schritt_3": "Modul_3", "Schritt_4": "Modul_4", "Schritt_5": "Modul_5", "Schritt_6": "Modul_6"}

In [13]:
# Anzahl der Werkstücke angeben

In [14]:
# Angabe der gewünschten Produktionsmenge der Produktarten
# Produktionsmenge = {"Produkt_1": 740, "Produkt_2": 600}
Produktionsmenge = {"Produkt_1": 148, "Produkt_2": 120}

# Produktionszeitraum (in h)
# Produktionszeitraum = 50
Produktionszeitraum = 10
# Betrachteten Zeitraum bestimmen
BZ = (Produktionszeitraum * 3600)

# Array der Outputs der Produkte definieren
tempout = [0]*len(Produktarten)
i=0
for Produkt in Produktarten:
    tempout[i] = int(Produktionsmenge[Produkt])
    i = i+1
    
# Anzahl Werkstücke der Produktarten bestimmen
AnzahlWerkstücke = {}
tempout2 = dict ((Produkt, Wert) for (Produkt, Wert) in zip(Produktarten, tempout))
for Produkt in Produktarten:
    AnzahlWerkstücke[Produkt] = tempout2[Produkt]

In [15]:
# Array der Werkstücke für alle Produktarten definieren
Werkstücke={}
for Produkt in Produktarten:
    Werkstücke[Produkt] = [0]*AnzahlWerkstücke[Produkt]
    for i in range(0,AnzahlWerkstücke[Produkt]):
        Werkstücke[Produkt][i] = "Werkstueck_" + str(i+1)

In [16]:
# Unsichere normalverteilte Bearbeitungszeiten

In [17]:
# Mittelwert und Standardabweichung der BZ der Schritte
Mittelwert = {"Schritt_1": 50, "Schritt_2": 250, "Schritt_3": 40, "Schritt_4": 180, "Schritt_5": 40, "Schritt_6": 60}
Standardabweichung = {"Schritt_1": 5, "Schritt_2": 25, "Schritt_3": 4, "Schritt_4": 18, "Schritt_5": 4, "Schritt_6": 6}

In [18]:
# gewünschte Sicherheit / Wahrscheinlichkeit mit der die Zeit eingehalten werden soll
# Übliche Werte für die Quantile sind z.B. 0.8; 0.85; 0.9; 0.95; 0.975; 0.99; 0.995; 0.999; 0.9995
Wahrscheinlichkeit = 0.85
# Bestimmung Φ (u_p)= Wahrscheinlichkeit der Standardnormalverteilung N(0, 1)
u_p = scipy.stats.norm.ppf(Wahrscheinlichkeit, loc=0, scale=1)

In [19]:
# Berechnung resultierende Bearbeitungszeiten, damit die Zeiten mit der angegebenen Wahrscheinlichkeit eingehalten werden
# x_p = μ + (σ * u_p)
Bearbeitungszeit = {}
for Produkt in Produktarten:
    x_p = [0]*len(Schritte[Produkt])
    i = -1 
    for Schritt in Schritte[Produkt]:
        i = i + 1
        x_p[i] = Mittelwert[Schritt] + (Standardabweichung[Schritt] * u_p)
    Bearbeitungszeit[Produkt] = dict((Schritt, Wert) for (Schritt, Wert) in zip(Schritte[Produkt], x_p))

In [20]:
# Unsichere exponentialverteilte Ausfallzeiten der Maschinen

In [21]:
# Festlegung der Ausfallkosten m €/s
m = 0.11
# Maximale Ausfallzeit [h]
MaximaleAusfallzeittemp = 3
# in Sekunden
MaximaleAusfallzeit = MaximaleAusfallzeittemp * 3600

In [22]:
# Definition der Ausfallraten je Maschine / Modul
# Ausfallrate Maschine
p = 0.0003
# Ausfallrate Module λ
λ_M1 = 0.0002
λ_M2 = 0.0002
λ_M3 = 0.0002
λ_M4 = 0.0002
λ_M5 = 0.0002
λ_M6 = 0.0002

In [23]:
# Berechnung der Erwartungswerte (durchschnittliche Zeit bis zum Ausfall) E(x)=1/λ
Ex_Maschine = 1/p
Ex_M1 = 1/λ_M1
Ex_M2 = 1/λ_M2
Ex_M3 = 1/λ_M3
Ex_M4 = 1/λ_M4
Ex_M5 = 1/λ_M5
Ex_M6 = 1/λ_M6

In [24]:
# Berechnung der erwarteten Anzahl an Fehlern = betrachteter Zeitraum / E(x)
Fehler_Maschine = BZ / Ex_Maschine
Fehler_Module = {"Modul_1": BZ / Ex_M1, "Modul_2": BZ / Ex_M2, "Modul_3": BZ / Ex_M3, "Modul_4": BZ / Ex_M4, "Modul_5": BZ / Ex_M5, "Modul_6": BZ / Ex_M6}

In [25]:
# Definition der durchschnittlichen Ausfalldauer einer Maschine / eines Moduls
Ausfalldauer_Maschine = 15
Ausfalldauer_Module = {"Modul_1": 10, "Modul_2": 10, "Modul_3": 10, "Modul_4": 10, "Modul_5": 10, "Modul_6": 10}

In [26]:
# Optimierungsmodell erstellen
model = gp.Model('MILP_Rekonfiguration')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-12


In [27]:
# Entscheidungsvariablen

In [28]:
# xwqik = 1, falls für Werkstück w von Produkt q Schritt i an Station k durchgeführt wird
x = {}
for Produkt in Produktarten:
    Werkstuck = Werkstücke[Produkt]
    Schrit = Bearbeitungszeit[Produkt]
    for Werkstück in Werkstuck:
        for Schritt in Schrit:
            for Station in Eröffnungskosten:
                x[Produkt, Werkstück, Schritt, Station] = model.addVar(vtype=GRB.BINARY, name="x[{},{},{},{}]".format(Produkt, Werkstück, Schritt, Station))

In [29]:
# zlk = 1, falls Prozessmodul l an Station k vorhanden ist
z = model.addVars(Anzahl, Eröffnungskosten, vtype=GRB.BINARY, name = "z")

In [30]:
# sk = 1, falls Station k vorhanden ist
s = model.addVars(Eröffnungskosten, vtype=GRB.BINARY, name = "s")

In [31]:
# ak: erwartete Ausfallzeit der Station k (≥0)
a = model.addVars(Eröffnungskosten, vtype=GRB.CONTINUOUS, name = "a")

In [32]:
# vk: Überschrittene Zeit an einer Station k (= (erwartete Gesamtzeit an einer Station (alle BZ + Ausfallzeiten)) - BZ) ≥0
v = model.addVars(Eröffnungskosten, vtype=GRB.CONTINUOUS, name = "v")

In [33]:
# Schlupfvariable um sicherzustellen, dass vorhandene Module nicht verkauft werden können
t = model.addVars(Anzahl, vtype=GRB.INTEGER, name = "t")

In [34]:
# Zielfunktion
objective = (sum(t[Modul] for Modul in Anzahl) + sum(Eröffnungskosten[Station]*s[Station] for Station in Eröffnungskosten) + sum(v[Station]*m for Station in Eröffnungskosten))
model.setObjective(objective, GRB.MINIMIZE)

In [35]:
# Nebenbedingungen

In [36]:
# Für jedes Werkstück w von Produkt q kann ein Schritt i nur dann an einer Station k durchgeführt werden, wenn diese auch vorhanden ist
for Produkt in Produktarten:
    for Werkstück in Werkstücke[Produkt]:
         for Schritt in Bearbeitungszeit[Produkt]:   
            for Station in Eröffnungskosten:
                model.addConstr(x[Produkt, Werkstück, Schritt, Station]-s[Station] <= 0, "für_{}_{}_{}_durchgeführt_an_{}".format(Produkt, Werkstück, Schritt, Station))

In [37]:
# Für jedes Werkstück w von Produkt q muss jeder für diese Produktart durchzuführende Schritt i an genau einer Station k durchgeführt werden
for Produkt in Produktarten:
    for Werkstück in Werkstücke[Produkt]:
        for Schritt in Bearbeitungszeit[Produkt]:
            model.addConstr(sum(x[Produkt, Werkstück, Schritt, Station] for Station in Eröffnungskosten) == 1, "für_{}_{}_wird_{}_durchgeführt".format(Produkt, Werkstück, Schritt))

In [38]:
# Wenn für ein Werkstück w von Produkt q Schritt i an einer Station k durchgeführt wird, so muss auch das dafür benötigte Prozessmodul l an der Station vorhanden sein
for Produkt in Produktarten:
    for Werkstück in Werkstücke[Produkt]:
        for Schritt in Bearbeitungszeit[Produkt]:
            for Station in Eröffnungskosten:
                model.addConstr((x[Produkt, Werkstück, Schritt, Station] - z[Prozessmodul[Schritt], Station] <= 0), "für_{}_{}_{}_vorhanden_an_{}".format(Produkt, Werkstück, Prozessmodul[Schritt], Station))

In [39]:
# Berechnung der erwarteten Ausfallzeit je Station im Betrachtungszeitraum (a_k)
for Station in Eröffnungskosten:
    model.addConstr((((s[Station]*Fehler_Maschine*Ausfalldauer_Maschine) + sum(z[Modul, Station]*Fehler_Module[Modul]*Ausfalldauer_Module[Modul] for Modul in Fehler_Module)) == a[Station]), "Berechnung_der_Ausfallzeit_von_{}".format(Station))

In [40]:
# Berechnung der überschrittenen Zeit je Station im Betrachtungszeitraum (v_k)
for Station in Eröffnungskosten:
    model.addConstr(((gp.quicksum(Bearbeitungszeit[Produkt][Schritt]*x[Produkt, Werkstück, Schritt, Station] for Produkt in Produktarten for Werkstück in Werkstücke[Produkt] for Schritt in Bearbeitungszeit[Produkt])) + a[Station] - BZ <= 0), "Berechnung_der_überschrittenen_Zeit_an_{}".format(Station))

In [41]:
# Berechnung der Kosten der Prozessmodule (>=0)
for Modul in Anzahl:
    model.addConstr((Kosten_Module(sum(z[Modul, Station] for Station in Eröffnungskosten), Modul) - t[Modul] <= 0), "Sicherstellen_Maximum_{}".format(Modul))

In [42]:
# Sicherstellen, dass die maximale Ausfallzeit eingehalten wird
model.addConstr((sum(v[Station] for Station in Eröffnungskosten) - MaximaleAusfallzeit <= 0), "Maximale_Ausfallzeit_einhalten")

<gurobi.Constr *Awaiting Model Update*>

In [43]:
# Optimierung
stopt = datetime.datetime.now()
model.optimize()
print(datetime.datetime.now()-stopt)

status = model.Status
if status == GRB.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
if status == GRB.OPTIMAL:
    print('The optimal objective is %g' % model.ObjVal)
if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 3 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 14319 rows, 6660 columns and 39702 nonzeros
Model fingerprint: 0xcd207eb9
Variable types: 12 continuous, 6648 integer (6642 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e-01, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+04]
Found heuristic solution: objective 210000.00000
Presolve removed 8 rows and 13 columns
Presolve time: 0.31s
Presolved: 14311 rows, 6647 columns, 39677 nonzeros
Variable types: 0 continuous, 6647 integer (6642 binary)

Root relaxation: objective 7.476747e+03, 11097 iterations, 2.65 seconds (2.76 work units)
Total elapsed time = 5.61s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 7476.74653    0 5534 210000.000 7476.74653  96.4%     

In [44]:
# Endzeit der Optimierung bestimmen
et = datetime.datetime.now()
# Ausführungsdauer berechnen
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 0:01:48.999739 seconds


In [45]:
# Ausgabe des Problems
print(model.display())

Minimize
<gurobi.LinExpr: 30000.0 s[Station_5] + 30000.0 s[Station_6] + 0.11 v[Station_1]
+ 0.11 v[Station_2] + 0.11 v[Station_3] + 0.11 v[Station_4] + 0.11 v[Station_5]
+ 0.11 v[Station_6] + t[Modul_1] + t[Modul_2] + t[Modul_3] + t[Modul_4] + t[Modul_5]
+ t[Modul_6]>
Subject To
für_Produkt_1_Werkstueck_1_Schritt_3_durchgeführt_an_Station_1: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_1,Schritt_3,Station_1] + -1.0 s[Station_1]> <= 0
für_Produkt_1_Werkstueck_1_Schritt_3_durchgeführt_an_Station_2: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_1,Schritt_3,Station_2] + -1.0 s[Station_2]> <= 0
für_Produkt_1_Werkstueck_1_Schritt_3_durchgeführt_an_Station_3: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_1,Schritt_3,Station_3] + -1.0 s[Station_3]> <= 0
für_Produkt_1_Werkstueck_1_Schritt_3_durchgeführt_an_Station_4: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_1,Schritt_3,Station_4] + -1.0 s[Station_4]> <= 0
für_Produkt_1_Werkstueck_1_Schritt_3_durchgeführt_an_Station_5: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck

 x[Produkt_1,Werkstueck_2,Schritt_2,Station_4] + -1.0 s[Station_4]> <= 0
für_Produkt_1_Werkstueck_2_Schritt_2_durchgeführt_an_Station_5: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_2,Schritt_2,Station_5] + -1.0 s[Station_5]> <= 0
für_Produkt_1_Werkstueck_2_Schritt_2_durchgeführt_an_Station_6: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_2,Schritt_2,Station_6] + -1.0 s[Station_6]> <= 0
für_Produkt_1_Werkstueck_2_Schritt_1_durchgeführt_an_Station_1: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_2,Schritt_1,Station_1] + -1.0 s[Station_1]> <= 0
für_Produkt_1_Werkstueck_2_Schritt_1_durchgeführt_an_Station_2: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_2,Schritt_1,Station_2] + -1.0 s[Station_2]> <= 0
für_Produkt_1_Werkstueck_2_Schritt_1_durchgeführt_an_Station_3: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_2,Schritt_1,Station_3] + -1.0 s[Station_3]> <= 0
für_Produkt_1_Werkstueck_2_Schritt_1_durchgeführt_an_Station_4: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_2,Schritt_1,Station_4] + -1.0 s[Station_4]> <= 0
für

für_Produkt_1_Werkstueck_4_Schritt_5_durchgeführt_an_Station_4: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_4,Schritt_5,Station_4] + -1.0 s[Station_4]> <= 0
für_Produkt_1_Werkstueck_4_Schritt_5_durchgeführt_an_Station_5: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_4,Schritt_5,Station_5] + -1.0 s[Station_5]> <= 0
für_Produkt_1_Werkstueck_4_Schritt_5_durchgeführt_an_Station_6: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_4,Schritt_5,Station_6] + -1.0 s[Station_6]> <= 0
für_Produkt_1_Werkstueck_4_Schritt_2_durchgeführt_an_Station_1: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_4,Schritt_2,Station_1] + -1.0 s[Station_1]> <= 0
für_Produkt_1_Werkstueck_4_Schritt_2_durchgeführt_an_Station_2: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_4,Schritt_2,Station_2] + -1.0 s[Station_2]> <= 0
für_Produkt_1_Werkstueck_4_Schritt_2_durchgeführt_an_Station_3: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_4,Schritt_2,Station_3] + -1.0 s[Station_3]> <= 0
für_Produkt_1_Werkstueck_4_Schritt_2_durchgeführt_an_Station_4: <gurobi.LinE

 x[Produkt_1,Werkstueck_6,Schritt_4,Station_3] + -1.0 s[Station_3]> <= 0
für_Produkt_1_Werkstueck_6_Schritt_4_durchgeführt_an_Station_4: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_6,Schritt_4,Station_4] + -1.0 s[Station_4]> <= 0
für_Produkt_1_Werkstueck_6_Schritt_4_durchgeführt_an_Station_5: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_6,Schritt_4,Station_5] + -1.0 s[Station_5]> <= 0
für_Produkt_1_Werkstueck_6_Schritt_4_durchgeführt_an_Station_6: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_6,Schritt_4,Station_6] + -1.0 s[Station_6]> <= 0
für_Produkt_1_Werkstueck_6_Schritt_5_durchgeführt_an_Station_1: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_6,Schritt_5,Station_1] + -1.0 s[Station_1]> <= 0
für_Produkt_1_Werkstueck_6_Schritt_5_durchgeführt_an_Station_2: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_6,Schritt_5,Station_2] + -1.0 s[Station_2]> <= 0
für_Produkt_1_Werkstueck_6_Schritt_5_durchgeführt_an_Station_3: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_6,Schritt_5,Station_3] + -1.0 s[Station_3]> <= 0
für

für_Produkt_1_Werkstueck_8_Schritt_3_durchgeführt_an_Station_3: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_8,Schritt_3,Station_3] + -1.0 s[Station_3]> <= 0
für_Produkt_1_Werkstueck_8_Schritt_3_durchgeführt_an_Station_4: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_8,Schritt_3,Station_4] + -1.0 s[Station_4]> <= 0
für_Produkt_1_Werkstueck_8_Schritt_3_durchgeführt_an_Station_5: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_8,Schritt_3,Station_5] + -1.0 s[Station_5]> <= 0
für_Produkt_1_Werkstueck_8_Schritt_3_durchgeführt_an_Station_6: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_8,Schritt_3,Station_6] + -1.0 s[Station_6]> <= 0
für_Produkt_1_Werkstueck_8_Schritt_4_durchgeführt_an_Station_1: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_8,Schritt_4,Station_1] + -1.0 s[Station_1]> <= 0
für_Produkt_1_Werkstueck_8_Schritt_4_durchgeführt_an_Station_2: <gurobi.LinExpr:
 x[Produkt_1,Werkstueck_8,Schritt_4,Station_2] + -1.0 s[Station_2]> <= 0
für_Produkt_1_Werkstueck_8_Schritt_4_durchgeführt_an_Station_3: <gurobi.LinE

...

'x[Produkt_2,Werkstueck_84,Schritt_2,Station_1]',
'x[Produkt_2,Werkstueck_84,Schritt_2,Station_2]',
'x[Produkt_2,Werkstueck_84,Schritt_2,Station_3]',
'x[Produkt_2,Werkstueck_84,Schritt_2,Station_4]',
'x[Produkt_2,Werkstueck_84,Schritt_2,Station_5]',
'x[Produkt_2,Werkstueck_84,Schritt_2,Station_6]',
'x[Produkt_2,Werkstueck_84,Schritt_6,Station_1]',
'x[Produkt_2,Werkstueck_84,Schritt_6,Station_2]',
'x[Produkt_2,Werkstueck_84,Schritt_6,Station_3]',
'x[Produkt_2,Werkstueck_84,Schritt_6,Station_4]',
'x[Produkt_2,Werkstueck_84,Schritt_6,Station_5]',
'x[Produkt_2,Werkstueck_84,Schritt_6,Station_6]',
'x[Produkt_2,Werkstueck_85,Schritt_4,Station_1]',
'x[Produkt_2,Werkstueck_85,Schritt_4,Station_2]',
'x[Produkt_2,Werkstueck_85,Schritt_4,Station_3]',
'x[Produkt_2,Werkstueck_85,Schritt_4,Station_4]',
'x[Produkt_2,Werkstueck_85,Schritt_4,Station_5]',
'x[Produkt_2,Werkstueck_85,Schritt_4,Station_6]',
'x[Produkt_2,Werkstueck_85,Schritt_2,Station_1]',
'x[Produkt_2,Werkstueck_85,Schritt_2,Station_2]',


'x[Produkt_2,Werkstueck_93,Schritt_2,Station_3]',
'x[Produkt_2,Werkstueck_93,Schritt_2,Station_4]',
'x[Produkt_2,Werkstueck_93,Schritt_2,Station_5]',
'x[Produkt_2,Werkstueck_93,Schritt_2,Station_6]',
'x[Produkt_2,Werkstueck_93,Schritt_6,Station_1]',
'x[Produkt_2,Werkstueck_93,Schritt_6,Station_2]',
'x[Produkt_2,Werkstueck_93,Schritt_6,Station_3]',
'x[Produkt_2,Werkstueck_93,Schritt_6,Station_4]',
'x[Produkt_2,Werkstueck_93,Schritt_6,Station_5]',
'x[Produkt_2,Werkstueck_93,Schritt_6,Station_6]',
'x[Produkt_2,Werkstueck_94,Schritt_4,Station_1]',
'x[Produkt_2,Werkstueck_94,Schritt_4,Station_2]',
'x[Produkt_2,Werkstueck_94,Schritt_4,Station_3]',
'x[Produkt_2,Werkstueck_94,Schritt_4,Station_4]',
'x[Produkt_2,Werkstueck_94,Schritt_4,Station_5]',
'x[Produkt_2,Werkstueck_94,Schritt_4,Station_6]',
'x[Produkt_2,Werkstueck_94,Schritt_2,Station_1]',
'x[Produkt_2,Werkstueck_94,Schritt_2,Station_2]',
'x[Produkt_2,Werkstueck_94,Schritt_2,Station_3]',
'x[Produkt_2,Werkstueck_94,Schritt_2,Station_4]',


'x[Produkt_2,Werkstueck_102,Schritt_2,Station_4]',
'x[Produkt_2,Werkstueck_102,Schritt_2,Station_5]',
'x[Produkt_2,Werkstueck_102,Schritt_2,Station_6]',
'x[Produkt_2,Werkstueck_102,Schritt_6,Station_1]',
'x[Produkt_2,Werkstueck_102,Schritt_6,Station_2]',
'x[Produkt_2,Werkstueck_102,Schritt_6,Station_3]',
'x[Produkt_2,Werkstueck_102,Schritt_6,Station_4]',
'x[Produkt_2,Werkstueck_102,Schritt_6,Station_5]',
'x[Produkt_2,Werkstueck_102,Schritt_6,Station_6]',
'x[Produkt_2,Werkstueck_103,Schritt_4,Station_1]',
'x[Produkt_2,Werkstueck_103,Schritt_4,Station_2]',
'x[Produkt_2,Werkstueck_103,Schritt_4,Station_3]',
'x[Produkt_2,Werkstueck_103,Schritt_4,Station_4]',
'x[Produkt_2,Werkstueck_103,Schritt_4,Station_5]',
'x[Produkt_2,Werkstueck_103,Schritt_4,Station_6]',
'x[Produkt_2,Werkstueck_103,Schritt_2,Station_1]',
'x[Produkt_2,Werkstueck_103,Schritt_2,Station_2]',
'x[Produkt_2,Werkstueck_103,Schritt_2,Station_3]',
'x[Produkt_2,Werkstueck_103,Schritt_2,Station_4]',
'x[Produkt_2,Werkstueck_103,Sch

'x[Produkt_2,Werkstueck_111,Schritt_2,Station_3]',
'x[Produkt_2,Werkstueck_111,Schritt_2,Station_4]',
'x[Produkt_2,Werkstueck_111,Schritt_2,Station_5]',
'x[Produkt_2,Werkstueck_111,Schritt_2,Station_6]',
'x[Produkt_2,Werkstueck_111,Schritt_6,Station_1]',
'x[Produkt_2,Werkstueck_111,Schritt_6,Station_2]',
'x[Produkt_2,Werkstueck_111,Schritt_6,Station_3]',
'x[Produkt_2,Werkstueck_111,Schritt_6,Station_4]',
'x[Produkt_2,Werkstueck_111,Schritt_6,Station_5]',
'x[Produkt_2,Werkstueck_111,Schritt_6,Station_6]',
'x[Produkt_2,Werkstueck_112,Schritt_4,Station_1]',
'x[Produkt_2,Werkstueck_112,Schritt_4,Station_2]',
'x[Produkt_2,Werkstueck_112,Schritt_4,Station_3]',
'x[Produkt_2,Werkstueck_112,Schritt_4,Station_4]',
'x[Produkt_2,Werkstueck_112,Schritt_4,Station_5]',
'x[Produkt_2,Werkstueck_112,Schritt_4,Station_6]',
'x[Produkt_2,Werkstueck_112,Schritt_2,Station_1]',
'x[Produkt_2,Werkstueck_112,Schritt_2,Station_2]',
'x[Produkt_2,Werkstueck_112,Schritt_2,Station_3]',
'x[Produkt_2,Werkstueck_112,Sch

'x[Produkt_2,Werkstueck_120,Schritt_2,Station_2]',
'x[Produkt_2,Werkstueck_120,Schritt_2,Station_3]',
'x[Produkt_2,Werkstueck_120,Schritt_2,Station_4]',
'x[Produkt_2,Werkstueck_120,Schritt_2,Station_5]',
'x[Produkt_2,Werkstueck_120,Schritt_2,Station_6]',
'x[Produkt_2,Werkstueck_120,Schritt_6,Station_1]',
'x[Produkt_2,Werkstueck_120,Schritt_6,Station_2]',
'x[Produkt_2,Werkstueck_120,Schritt_6,Station_3]',
'x[Produkt_2,Werkstueck_120,Schritt_6,Station_4]',
'x[Produkt_2,Werkstueck_120,Schritt_6,Station_5]',
'x[Produkt_2,Werkstueck_120,Schritt_6,Station_6]', 'z[Modul_1,Station_1]',
'z[Modul_1,Station_2]', 'z[Modul_1,Station_3]', 'z[Modul_1,Station_4]',
'z[Modul_1,Station_5]', 'z[Modul_1,Station_6]', 'z[Modul_2,Station_1]',
'z[Modul_2,Station_2]', 'z[Modul_2,Station_3]', 'z[Modul_2,Station_4]',
'z[Modul_2,Station_5]', 'z[Modul_2,Station_6]', 'z[Modul_3,Station_1]',
'z[Modul_3,Station_2]', 'z[Modul_3,Station_3]', 'z[Modul_3,Station_4]',
'z[Modul_3,Station_5]', 'z[Modul_3,Station_6]', 'z[Modu

In [46]:
# Ausgabe der Lösung
all_vars = model.getVars()
values = model.getAttr("X", all_vars)
names = model.getAttr("VarName", all_vars)

for name, val in zip(names, values):
    print(f"{name} = {val}")

x[Produkt_1,Werkstueck_1,Schritt_3,Station_1] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_3,Station_2] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_3,Station_3] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_3,Station_4] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_3,Station_5] = 1.0
x[Produkt_1,Werkstueck_1,Schritt_3,Station_6] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_4,Station_1] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_4,Station_2] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_4,Station_3] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_4,Station_4] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_4,Station_5] = 1.0
x[Produkt_1,Werkstueck_1,Schritt_4,Station_6] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_5,Station_1] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_5,Station_2] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_5,Station_3] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_5,Station_4] = 1.0
x[Produkt_1,Werkstueck_1,Schritt_5,Station_5] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_5,Station_6] = -0.0
x[Produkt_1,Werkstueck_1,Schritt_2,Station_1] = -

x[Produkt_1,Werkstueck_12,Schritt_1,Station_1] = -0.0
x[Produkt_1,Werkstueck_12,Schritt_1,Station_2] = 1.0
x[Produkt_1,Werkstueck_12,Schritt_1,Station_3] = 0.0
x[Produkt_1,Werkstueck_12,Schritt_1,Station_4] = -0.0
x[Produkt_1,Werkstueck_12,Schritt_1,Station_5] = -0.0
x[Produkt_1,Werkstueck_12,Schritt_1,Station_6] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_3,Station_1] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_3,Station_2] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_3,Station_3] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_3,Station_4] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_3,Station_5] = 1.0
x[Produkt_1,Werkstueck_13,Schritt_3,Station_6] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_4,Station_1] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_4,Station_2] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_4,Station_3] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_4,Station_4] = -0.0
x[Produkt_1,Werkstueck_13,Schritt_4,Station_5] = 1.0
x[Produkt_1,Werkstueck_13,Schritt_4,Station_6] = -0.0
x[Produkt_1,Werkstueck_13,Schrit

x[Produkt_1,Werkstueck_29,Schritt_5,Station_2] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_5,Station_3] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_5,Station_4] = 1.0
x[Produkt_1,Werkstueck_29,Schritt_5,Station_5] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_5,Station_6] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_2,Station_1] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_2,Station_2] = 1.0
x[Produkt_1,Werkstueck_29,Schritt_2,Station_3] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_2,Station_4] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_2,Station_5] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_2,Station_6] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_1,Station_1] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_1,Station_2] = 1.0
x[Produkt_1,Werkstueck_29,Schritt_1,Station_3] = 0.0
x[Produkt_1,Werkstueck_29,Schritt_1,Station_4] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_1,Station_5] = -0.0
x[Produkt_1,Werkstueck_29,Schritt_1,Station_6] = -0.0
x[Produkt_1,Werkstueck_30,Schritt_3,Station_1] = -0.0
x[Produkt_1,Werkstueck_30,Schrit

x[Produkt_1,Werkstueck_36,Schritt_3,Station_1] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_3,Station_2] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_3,Station_3] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_3,Station_4] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_3,Station_5] = 1.0
x[Produkt_1,Werkstueck_36,Schritt_3,Station_6] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_4,Station_1] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_4,Station_2] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_4,Station_3] = 1.0
x[Produkt_1,Werkstueck_36,Schritt_4,Station_4] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_4,Station_5] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_4,Station_6] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_5,Station_1] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_5,Station_2] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_5,Station_3] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_5,Station_4] = 1.0
x[Produkt_1,Werkstueck_36,Schritt_5,Station_5] = -0.0
x[Produkt_1,Werkstueck_36,Schritt_5,Station_6] = -0.0
x[Produkt_1,Werkstueck_36,Schri

x[Produkt_1,Werkstueck_46,Schritt_3,Station_3] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_3,Station_4] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_3,Station_5] = 1.0
x[Produkt_1,Werkstueck_46,Schritt_3,Station_6] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_4,Station_1] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_4,Station_2] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_4,Station_3] = 1.0
x[Produkt_1,Werkstueck_46,Schritt_4,Station_4] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_4,Station_5] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_4,Station_6] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_5,Station_1] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_5,Station_2] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_5,Station_3] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_5,Station_4] = 1.0
x[Produkt_1,Werkstueck_46,Schritt_5,Station_5] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_5,Station_6] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_2,Station_1] = -0.0
x[Produkt_1,Werkstueck_46,Schritt_2,Station_2] = -0.0
x[Produkt_1,Werkstueck_46,Schri

...

x[Produkt_2,Werkstueck_23,Schritt_2,Station_6] = -0.0
x[Produkt_2,Werkstueck_23,Schritt_6,Station_1] = -0.0
x[Produkt_2,Werkstueck_23,Schritt_6,Station_2] = -0.0
x[Produkt_2,Werkstueck_23,Schritt_6,Station_3] = -0.0
x[Produkt_2,Werkstueck_23,Schritt_6,Station_4] = -0.0
x[Produkt_2,Werkstueck_23,Schritt_6,Station_5] = 1.0
x[Produkt_2,Werkstueck_23,Schritt_6,Station_6] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_4,Station_1] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_4,Station_2] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_4,Station_3] = 1.0
x[Produkt_2,Werkstueck_24,Schritt_4,Station_4] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_4,Station_5] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_4,Station_6] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_2,Station_1] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_2,Station_2] = 1.0
x[Produkt_2,Werkstueck_24,Schritt_2,Station_3] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_2,Station_4] = -0.0
x[Produkt_2,Werkstueck_24,Schritt_2,Station_5] = -0.0
x[Produkt_2,Werkstueck_24,Schri

x[Produkt_2,Werkstueck_37,Schritt_6,Station_2] = -0.0
x[Produkt_2,Werkstueck_37,Schritt_6,Station_3] = -0.0
x[Produkt_2,Werkstueck_37,Schritt_6,Station_4] = -0.0
x[Produkt_2,Werkstueck_37,Schritt_6,Station_5] = 1.0
x[Produkt_2,Werkstueck_37,Schritt_6,Station_6] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_4,Station_1] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_4,Station_2] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_4,Station_3] = 1.0
x[Produkt_2,Werkstueck_38,Schritt_4,Station_4] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_4,Station_5] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_4,Station_6] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_2,Station_1] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_2,Station_2] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_2,Station_3] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_2,Station_4] = 1.0
x[Produkt_2,Werkstueck_38,Schritt_2,Station_5] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_2,Station_6] = -0.0
x[Produkt_2,Werkstueck_38,Schritt_6,Station_1] = -0.0
x[Produkt_2,Werkstueck_38,Schri

x[Produkt_2,Werkstueck_51,Schritt_2,Station_1] = 1.0
x[Produkt_2,Werkstueck_51,Schritt_2,Station_2] = 0.0
x[Produkt_2,Werkstueck_51,Schritt_2,Station_3] = -0.0
x[Produkt_2,Werkstueck_51,Schritt_2,Station_4] = -0.0
x[Produkt_2,Werkstueck_51,Schritt_2,Station_5] = -0.0
x[Produkt_2,Werkstueck_51,Schritt_2,Station_6] = -0.0
x[Produkt_2,Werkstueck_51,Schritt_6,Station_1] = -0.0
x[Produkt_2,Werkstueck_51,Schritt_6,Station_2] = -0.0
x[Produkt_2,Werkstueck_51,Schritt_6,Station_3] = -0.0
x[Produkt_2,Werkstueck_51,Schritt_6,Station_4] = -0.0
x[Produkt_2,Werkstueck_51,Schritt_6,Station_5] = 1.0
x[Produkt_2,Werkstueck_51,Schritt_6,Station_6] = -0.0
x[Produkt_2,Werkstueck_52,Schritt_4,Station_1] = -0.0
x[Produkt_2,Werkstueck_52,Schritt_4,Station_2] = -0.0
x[Produkt_2,Werkstueck_52,Schritt_4,Station_3] = 1.0
x[Produkt_2,Werkstueck_52,Schritt_4,Station_4] = -0.0
x[Produkt_2,Werkstueck_52,Schritt_4,Station_5] = -0.0
x[Produkt_2,Werkstueck_52,Schritt_4,Station_6] = -0.0
x[Produkt_2,Werkstueck_52,Schrit

x[Produkt_2,Werkstueck_79,Schritt_4,Station_3] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_4,Station_4] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_4,Station_5] = 1.0
x[Produkt_2,Werkstueck_79,Schritt_4,Station_6] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_2,Station_1] = 1.0
x[Produkt_2,Werkstueck_79,Schritt_2,Station_2] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_2,Station_3] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_2,Station_4] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_2,Station_5] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_2,Station_6] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_6,Station_1] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_6,Station_2] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_6,Station_3] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_6,Station_4] = -0.0
x[Produkt_2,Werkstueck_79,Schritt_6,Station_5] = 1.0
x[Produkt_2,Werkstueck_79,Schritt_6,Station_6] = -0.0
x[Produkt_2,Werkstueck_80,Schritt_4,Station_1] = -0.0
x[Produkt_2,Werkstueck_80,Schritt_4,Station_2] = -0.0
x[Produkt_2,Werkstueck_80,Schri

x[Produkt_2,Werkstueck_106,Schritt_6,Station_4] = -0.0
x[Produkt_2,Werkstueck_106,Schritt_6,Station_5] = 1.0
x[Produkt_2,Werkstueck_106,Schritt_6,Station_6] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_4,Station_1] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_4,Station_2] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_4,Station_3] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_4,Station_4] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_4,Station_5] = 1.0
x[Produkt_2,Werkstueck_107,Schritt_4,Station_6] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_2,Station_1] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_2,Station_2] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_2,Station_3] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_2,Station_4] = 1.0
x[Produkt_2,Werkstueck_107,Schritt_2,Station_5] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_2,Station_6] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_6,Station_1] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_6,Station_2] = -0.0
x[Produkt_2,Werkstueck_107,Schritt_6,Station_3] = -0.0
x[Produkt_2,W

In [47]:
# Ausgabe des Problems in LP-Datei
model.write("MILP.lp")

In [48]:
# Ausgabe des Problems in MPS-Datei
model.write("MILP.mps")